# A Stochastic Dynamic Programming Model for Post-Disaster Relief Delivery considering the Deprivation Level

### 0. Install packages

In [ ]:
# Library for processing, analyzing, and displaying data
import pandas as pd

# Library for dictionary subclass that remembers the order in which keys were first inserted
from collections import OrderedDict

# Library for the creation, manipulation, and study of complex networks of nodes and edges
import networkx as nx

# Library used for creating static, interactive, and animated visualizations
import matplotlib.pyplot as plt

# Library for calculus
import numpy as np

# Library for finding a solution to the system of equations
from scipy.optimize import fsolve

# Library for data visualization
import seaborn as sns

### 1. Read arc data

In [ ]:
def load_data(file_path: str):
    df = pd.read_excel(file_path)
    df = df.astype({
        "from_node": int,
        "to_node": int,
        "t_1": float,
        "t_2": float,
        "t_3": float,
        "p_1": float,
        "p_2": float,
        "p_3": float
    })
    return df

file_path = "/Users/minhthipham/Library/CloudStorage/OneDrive-ErasmusUniversityRotterdam/Thesis/04_Simulation network/Arc data_v2.xlsx"
df = load_data(file_path)

### 2. Create a graph and map paths to arcs

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add edges (arcs) to the graph
G.add_edges_from([(0, 1), (0, 2), (0, 3),
                  (1, 4), (1, 5), (1, 6),
                  (2, 4), (2, 5), (2, 6),
                  (3, 4), (3, 5), (3, 6),
                  (4, 7), (4, 8), (4, 9),
                  (5, 7), (5, 8), (5, 9),
                  (6, 7), (6, 8), (6, 9),
                  (7, 10), (7, 11), (7, 12),
                  (8, 10), (8, 11), (8, 12),
                  (9, 10), (9, 11), (9, 12),
                  (10, 13),
                  (11, 13),
                  (12, 13)])

# Define node positions
positions = {0: (0, 1), 1: (1, 2), 2: (1, 1), 3: (1, 0),
       4: (2, 2), 5: (2, 1), 6: (2, 0),
       7: (3, 2), 8: (3, 1), 9: (3, 0),
       10: (4, 2), 11: (4, 1), 12: (4, 0),
       13: (5, 1)}

# Plot the graph
nx.draw(G, positions, with_labels=True, node_size=1500, node_color='lightgrey', font_weight='bold', arrowsize=20)
plt.show()

# Find all paths from source to target
all_paths = list(nx.all_simple_paths(G, source=0, target=max(G.nodes)))


### 3. Nodes and their descendants

In [ ]:
# Reverse the graph to calculate predecessors
G_reversed = G.reverse()
descendants_dict = OrderedDict((node, list(G_reversed.predecessors(node))) for node in G_reversed.nodes())

### 4. All possible states

In [ ]:
# Precompute deprivation levels for a relevant range of times
unique_times = set()
def calculate_states(df):
    for _, row in df.iterrows():
        from_node = int(row['from_node'])
        to_node = int(row['to_node'])
        t_1 = row['t_1']
        t_2 = row['t_2']
        t_3 = row['t_3']

        if from_node in states_dict:
            if to_node not in states_dict:
                states_dict[to_node] = []
            for state in states_dict[from_node]:
                current_time = state[1]
                new_time_1 = round(current_time + t_1, 4)
                new_time_2 = round(current_time + t_2, 4)
                new_time_3 = round(current_time + t_3, 4)
                states_dict[to_node].append((to_node, new_time_1))
                states_dict[to_node].append((to_node, new_time_2))
                states_dict[to_node].append((to_node, new_time_3))
                unique_times.update([new_time_1, new_time_2, new_time_3])

states_dict = OrderedDict([(0, [(0, 0)])]) # Initialize states_dict
calculate_states(df)


### 5. Define Deprivation Level Function

In [ ]:
# Define the deprivation level function
def deprivation_level_function(x):
    return 9.772697 / (1 + 3.9031 * np.exp(-0.7919 * x))

### 6. Inflection point

In [ ]:
# Define the original function and its derivatives
def Y(X):
    return 9.772697 / (1 + 3.9031 * np.exp(-0.7919 * X))

def dY_dX(X):
    A = 9.772697
    B = 3.9031
    k = 0.7919
    return (A * k * B * np.exp(-k * X)) / ((1 + B * np.exp(-k * X)) ** 2)

def d2Y_dX2(X):
    A = 9.772697
    B = 3.9031
    k = 0.7919
    term1 = (-A * k**2 * B * np.exp(-k * X)) / ((1 + B * np.exp(-k * X)) ** 2)
    term2 = (2 * A * k**2 * B**2 * np.exp(-2 * k * X)) / ((1 + B * np.exp(-k * X)) ** 3)
    return term1 + term2

# Generate X values
X_values = np.linspace(0, 10, 400)

# Calculate Y, first derivative, and second derivative values
Y_values = Y(X_values)
dY_dX_values = dY_dX(X_values)
d2Y_dX2_values = d2Y_dX2(X_values)

# Find the X value where the first derivative has its maximum
max_dY_dX_index = np.argmax(dY_dX_values)
max_dY_dX_X_value = X_values[max_dY_dX_index]

# Find the inflection point (where second derivative equals zero)
inflection_point = fsolve(d2Y_dX2, 2)[0]

# Set font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'

# Plot the original function
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(X_values, Y_values, label='Y(X)', color='black')
plt.axvline(x=inflection_point, color='#5A6F97', linestyle='--', label='Inflection Point')
plt.title('Deprivation Level over Time')
plt.xlabel('Deprivation Time (days)')
plt.ylabel('Deprivation Level')
plt.xlim(0, 8)
plt.grid(True, which='both', linewidth=0.5, color='lightgrey')
plt.legend()

# Plot the first derivative
plt.subplot(3, 1, 2)
plt.plot(X_values, dY_dX_values, label="dY/dX", color='black')
plt.axvline(x=inflection_point, color='#5A6F97', linestyle='--', label='Inflection Point')
plt.title('Rate of Change of Deprivation Level')
plt.xlabel('Deprivation Time (days)')
plt.ylabel('dY/dX')
plt.xlim(0, 8)
plt.grid(True, which='both', linewidth=0.5, color='lightgrey')
plt.legend()

# Plot the second derivative
plt.subplot(3, 1, 3)
plt.plot(X_values, d2Y_dX2_values, label="d2Y/dX2", color='black')
plt.axvline(x=inflection_point, color='#5A6F97', linestyle='--', label='Inflection Point')
plt.title('Acceleration of Change of Deprivation Level')
plt.xlabel('Deprivation Time (days)')
plt.ylabel('d2Y/dX2')
plt.xlim(0, 8)
plt.grid(True, which='both', linewidth=0.5, color='lightgrey')
plt.legend()

# Adjust the space between plots
plt.subplots_adjust(hspace=0.5)  # Adjust the hspace value for better spacing

plt.tight_layout()
plt.savefig('Derivatives_DLF.pdf', format='pdf', bbox_inches='tight', dpi=300)
plt.show()

### 7. Recursion

In [ ]:
# Helper function to get transition probabilities and times
def get_transition_values(n, d):
    row = df.loc[(df["from_node"] == n) & (df["to_node"] == d)]
    p_1 = row["p_1"].values[0]
    p_2 = row["p_2"].values[0]
    p_3 = row["p_3"].values[0]
    t_1 = row["t_1"].values[0]
    t_2 = row["t_2"].values[0]
    t_3 = row["t_3"].values[0]
    return p_1, p_2, p_3, t_1, t_2, t_3

In [ ]:
# Determine the optimal policy
policy = {}
v_dict = OrderedDict()
v_dict_all = OrderedDict()
final_node = max(G.nodes)

for n, states in reversed(states_dict.items()):
    for state in states:
        tau = state[1]
        if n == final_node:
            v_dict[state] = deprivation_level_function(tau)
        else:
            min_V = np.inf
            best_d = None
            for d in descendants_dict.get(n, []):
                p_1, p_2, p_3, t_1, t_2, t_3 = get_transition_values(n, d)
                v_d_1 = v_dict.get((d, round(tau + t_1, 4)))
                v_d_2 = v_dict.get((d, round(tau + t_2, 4)))
                v_d_3 = v_dict.get((d, round(tau + t_3, 4)))
                V = p_1 * v_d_1 + p_2 * v_d_2 + p_3 * v_d_3
                v_dict_all[(n, tau, d)] = V
                if V < min_V:
                    min_V = V
                    best_d = d
            # cache deprivation levels
            v_dict[state] = min_V
            if best_d is not None:
                policy[state] = {
                    "best_next_node": best_d, 
                    "total_expected_deprivation_level": min_V
                }


### 8. Final policy

In [ ]:
import copy

# Determine the optimal policy
policy_final = copy.deepcopy(policy)
for state, _ in policy_final.items():
    n = state[0]
    tau = state[1]
    min_t_1 = np.inf
    if tau >= inflection_point:
        for d in descendants_dict.get(n, []):
            p_1, _, _, t_1, _, _ = get_transition_values(n, d)
            if t_1 < min_t_1:
                min_t_1 = t_1
                min_p_1 = p_1
                best_d = d
            elif t_1 == min_t_1 and p_1 > min_p_1: # if any descendants have the same t_1 choose the one with the higher probability
                min_t_1 = t_1
                min_p_1 = p_1
                best_d = d
        policy_final[state]["best_next_node"] = best_d
        policy_final[state]["total_expected_deprivation_level"] = v_dict_all[n, tau, best_d]


In [ ]:
policy_df = pd.DataFrame.from_dict(data=policy_final, orient='index')

In [ ]:
# Transforming the dictionary into a DataFrame
data = []
for key, value in policy_final.items():
    row = {
        'state': key,
        'best_next_node': value['best_next_node'],
        'total_expected_deprivation_level': value['total_expected_deprivation_level']
    }
    data.append(row)

df_out = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df_out.to_excel('output_v2.xlsx', index=False)

### 9. Expected Shortest Path

In [ ]:
# Create a directed graph
G_2 = nx.DiGraph()

# Add edges with expected travel time as weights
for _, row in df.iterrows():
    expected_travel_time = round(row['t_1'] * row['p_1'] + row['t_2'] * row['p_2'] + row['t_3'] * row['p_3'], 4)
    G_2.add_edge(row['from_node'], row['to_node'], weight=expected_travel_time)

# Compute the shortest path based on expected travel time
source = 0
target = max(G_2.nodes)
shortest_path = nx.shortest_path(G_2, source=source, target=target, weight='weight')
shortest_path_length = nx.shortest_path_length(G_2, source=source, target=target, weight='weight')
shortest_path_DL = deprivation_level_function(shortest_path_length)

# Display results
print(f"The shortest path from node {source} to node {target} is: {shortest_path}")
print(f"The total expected travel time is: {shortest_path_length} days")
print(f"The total expected deprivation level is: {shortest_path_DL}")

In [ ]:
# Create a directed graph
G_2 = nx.DiGraph()

# Add edges with expected travel time as weights
for _, row in df.iterrows():
    expected_travel_time = round(row['t_1'] * row['p_1'] + row['t_2'] * row['p_2'] + row['t_3'] * row['p_3'], 4)
    G_2.add_edge(row['from_node'], row['to_node'], weight=expected_travel_time)

# Define source and target nodes for the shortest path calculation
source_node = 0
target_node = max(G_2.nodes)

# Apply Dijkstra's algorithm to find the shortest path and its length
shortest_path = nx.dijkstra_path(G_2, source=source_node, target=target_node)
shortest_path_length = nx.dijkstra_path_length(G_2, source=source_node, target=target_node)

# Initialize the table with infinity distances
nodes = list(G_2.nodes)
table = pd.DataFrame(index=nodes, columns=['Distance', 'Previous Node'])
table['Distance'] = float('inf')
table['Previous Node'] = None

# Function to update the table
def update_table(node, distance, previous_node):
    table.at[node, 'Distance'] = distance
    table.at[node, 'Previous Node'] = previous_node

# Initialize the source node
update_table(source_node, 0, '-')

# Dijkstra's algorithm steps
steps_table_columns = ['Step', 'Current Node', 'Distance from Source', 'Neighbors', 'Distance Update', 'Updated Node Distances', 'Unvisited Nodes']
steps_table = pd.DataFrame(columns=steps_table_columns)

unvisited_nodes = set(nodes)
current_node = source_node
step = 1

while unvisited_nodes:
    current_distance = table.loc[current_node, 'Distance']
    neighbors = list(G_2.neighbors(current_node))
    distance_update = {}
    for neighbor in neighbors:
        if neighbor in unvisited_nodes:
            edge_weight = G_2[current_node][neighbor]['weight']
            new_distance = current_distance + edge_weight
            if new_distance < table.loc[neighbor, 'Distance']:
                update_table(neighbor, new_distance, current_node)
                distance_update[neighbor] = f"{current_distance}+{edge_weight}={new_distance}"
            else:
                distance_update[neighbor] = f"{table.loc[neighbor, 'Distance']}"
    updated_node_distances = dict(table['Distance'])
    steps_table.loc[step] = [step, current_node, current_distance, neighbors, distance_update, updated_node_distances, unvisited_nodes.copy()]
    unvisited_nodes.remove(current_node)
    if not unvisited_nodes:
        break
    current_node = table.loc[list(unvisited_nodes), 'Distance'].idxmin()
    step += 1

# Validate the results using NetworkX's built-in Dijkstra's algorithm
# NetworkX implementation of Dijkstra's algorithm for validation
nx_shortest_path = nx.dijkstra_path(G_2, source=source_node, target=target_node)
nx_shortest_path_length = nx.dijkstra_path_length(G_2, source=source_node, target=target_node)

# Validate the shortest path and its length
validation_result = (nx_shortest_path, nx_shortest_path_length) == (shortest_path, shortest_path_length)
validation_result, nx_shortest_path, nx_shortest_path_length, shortest_path, shortest_path_length

# Save table in Excel file
steps_table.to_excel('steps_table_2.xlsx', index=False)

### 10. Comparison

#### 10.1 Get all first states that deviate from the shorest path

In [ ]:
# Function to find all unique paths starting from a given node
def find_unique_paths(df, start_node, end_node):
    paths = set()
    
    def traverse_path(node, path):
        path = path + [node]
        
        if node == end_node:
            paths.add(tuple(path))  # Convert list to tuple to ensure uniqueness in a set
            return
        
        # Find the next node based on the current node
        next_nodes = df.loc[(node, slice(None)), 'best_next_node'].tolist()
        
        for next_node in next_nodes:
            traverse_path(next_node, path)
    
    traverse_path(start_node, [])
    return [list(p) for p in paths]  # Convert each path back to a list

# Assuming we start at node 0 and end at node 9
start_node = 0
end_node = final_node

unique_paths = find_unique_paths(policy_df, start_node, end_node)

divergent_states = []  # List to store all divergent nodes with details
for path in unique_paths:
    for i, (n_s, n_p) in enumerate(zip(shortest_path, path)):
        if int(n_s) != int(n_p):  # Check if nodes diverge
            # Once a divergence is found, look for the corresponding state in the policy
            for key, val in policy_final.items():
                current_node = key[0]
                elapsed_time = key[1]
                best_next_node = val["best_next_node"]

                # Check if this policy state aligns with the diverging path
                if current_node == path[i-1]:  # Match the current node with the previous node in the path
                    if int(best_next_node) == int(n_p):  # Match the best next node with the diverging path node
                        divergent_states.append({
                            "starting_node": current_node,
                            "elapsed_time": elapsed_time
                        })
            break  # Exit inner loop after finding first divergence in the current path


#### 10.2 Simulation

In [ ]:
# Function to run Monte Carlo simulation for a single start entry
def run_simulation_for_start(policy_final, shortest_path, start_entry, num_runs):
    policy_results = []
    shortest_path_results = []

    for _ in range(num_runs):
        starting_node = start_entry['starting_node']
        elapsed_time = start_entry['elapsed_time']

        # Simulate travel using the optimal policy
        current_node = starting_node
        total_time_policy = elapsed_time
        path_policy = [current_node]

        while current_node != final_node:
            next_node = policy_final[(current_node, total_time_policy)]["best_next_node"]
            p_1, p_2, p_3, t_1, t_2, t_3 = get_transition_values(current_node, next_node)
            
            # Simulate the travel scenario
            travel_time = np.random.choice([t_1, t_2, t_3], p=[p_1, p_2, p_3])
            
            total_time_policy += travel_time
            total_time_policy = round(total_time_policy, 4)
            current_node = next_node
            path_policy.append(current_node)
        
        realized_deprivation_level_policy = deprivation_level_function(total_time_policy)
        policy_results.append({
            "realized_deprivation_level": realized_deprivation_level_policy,
            "total_time": total_time_policy,
            "elapsed_time_node6": str(elapsed_time),
        })

        # Simulate travel using the shortest path
        total_time_shortest_path = elapsed_time
        for i in range(shortest_path.index(starting_node), len(shortest_path) - 1):
            from_node = shortest_path[i]
            to_node = shortest_path[i + 1]
            p_1, p_2, p_3, t_1, t_2, t_3 = get_transition_values(from_node, to_node)
            
            # Simulate the travel time scenario
            travel_time = np.random.choice([t_1, t_2, t_3], p=[p_1, p_2, p_3])
            total_time_shortest_path += travel_time
    
        # Calculate the realized deprivation level
        realized_deprivation_level_shortest_path = deprivation_level_function(total_time_shortest_path)
        shortest_path_results.append({
            "realized_deprivation_level": realized_deprivation_level_shortest_path,
            "total_time": total_time_shortest_path,
            "elapsed_time_node6": str(elapsed_time),
        })
    
    return policy_results, shortest_path_results

# Function to run Monte Carlo simulation for all start entries
def monte_carlo_simulation(policy_final, shortest_path, start_data_list, num_runs):
    all_policy_results = []
    all_shortest_path_results = []

    for start_entry in start_data_list:
        policy_results, shortest_path_results = run_simulation_for_start(policy_final, shortest_path, start_entry, num_runs)
        all_policy_results.extend(policy_results)
        all_shortest_path_results.extend(shortest_path_results)
    
    return all_policy_results, all_shortest_path_results

# Run the Monte Carlo simulation
num_runs = 1000
policy_results, shortest_path_results = monte_carlo_simulation(policy_final, shortest_path, divergent_states, num_runs)


#### 10.3 Visualization

In [ ]:
# Convert results to DataFrame for analysis
policy_results_df = pd.DataFrame(policy_results, columns=['realized_deprivation_level', 'total_time', 'elapsed_time_node6'])
shortest_path_results_df = pd.DataFrame(shortest_path_results, columns=['realized_deprivation_level', 'total_time', 'elapsed_time_node6'])

# Statistical analysis
policy_mean_deprivation = policy_results_df['realized_deprivation_level'].mean()
shortest_mean_deprivation = shortest_path_results_df['realized_deprivation_level'].mean()

policy_mean_time = policy_results_df['total_time'].mean()
shortest_mean_time = shortest_path_results_df['total_time'].mean()

print(f"Policy Mean Deprivation: {policy_mean_deprivation}")
print(f"Shortest Path Mean Deprivation: {shortest_mean_deprivation}")
print(f"Policy Mean Travel Time: {policy_mean_time}")
print(f"Shortest Path Mean Travel Time: {shortest_mean_time}")


In [ ]:
# Combined Visualization
plt.figure(figsize=(12, 4))

# First subplot
plt.subplot(1, 2, 1)
plt.hist(policy_results_df['realized_deprivation_level'], bins=80, alpha=0.7, label='Policy', color='#012D63', edgecolor='black')
plt.xlabel('Realized Deprivation Level')
plt.ylabel('Frequency')
plt.legend()

# Second subplot
plt.subplot(1, 2, 2)
plt.hist(shortest_path_results_df['realized_deprivation_level'], bins=80, alpha=0.7, label='Shortest Path', color='grey', edgecolor='black')
plt.xlabel('Realized Deprivation Level')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.savefig('MCS_results_v2.pdf', format='pdf', bbox_inches='tight', dpi=300)
plt.show()

#### 10.4 Statistics

In [ ]:
policy_results_df.describe()

In [ ]:
shortest_path_results_df.describe()